In [2]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#2  ваша задача – найти все произведения, связанные с подсолнухами. 
# Выполните поиск по слову "sunflowers". 
# Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.
# В качестве ответа выберите id объектов (3), которые присутствуют в полученном списке:

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
res = r.json()

In [22]:
sunflower_ids = res[ 'objectIDs']

search = [16822570, 436524, 2032, 437329, 20149]
find_id = []
for i in search:
    if i in sunflower_ids:
        find_id += [i]
find_id

[436524, 2032, 437329]

In [5]:
# 3 Выполните запрос для объекта 437980. Для проверки соотнесите полученные параметры с их значениями:
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')
obj_test = r_test.json()

In [9]:
obj_test['objectID']

437980

In [21]:
#4 Теперь напишите цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. 
# Результаты запишите в датафрейм, а именно – следующие параметры:

# objectID – id объекта
# title – название
# artistDisplayName – автор
# department – департамент
# objectBeginDate – дата (начало)
# objectEndDate – дата (конец)
# period – название периода
# objectName – название/категория объекта
# culture – культура

data=[] #создаем пустой список, из которого будем потом собирать DF

for i in sunflower_ids:
    r_find = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
    obj_find = r_find.json()
    obj_data = []
    try: data +=[ [obj_find['objectID'], 
                      obj_find['title'], 
                      obj_find['artistDisplayName'], 
                      obj_find['department'], 
                      obj_find['objectBeginDate'], 
                      obj_find['objectEndDate'], 
                      obj_find['period'], 
                      obj_find['objectName'], 
                      obj_find['culture']]]
    except: 
        continue
df =pd.DataFrame(data, columns = ['objectID', 
                                  'title', 
                                  'artistDisplayName', 
                                  'department',
                                 'objectBeginDate',
                                 'objectEndDate',
                                 'period',
                                 'objectName', 
                                 'culture'])
df

,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,436524,Sunflowers,Vincent van Gogh,European Paintings,1887,1887,,Painting,
1,484935,Sunflowers,Edward McKnight Kauffer,Modern and Contemporary Art,1921,1921,,Painting,
2,437112,Bouquet of Sunflowers,Claude Monet,European Paintings,1881,1881,,Painting,
3,210191,Vase with sunflowers,Rozenburg Plateelfabriek,European Sculpture and Decorative Arts,1896,1896,,Vase,
4,431264,Sunflowers,Lily Converse,Drawings and Prints,1941,1941,,Print,
...,...,...,...,...,...,...,...,...,...
79,436144,James-Jacques-Joseph Tissot (1836–1902),Edgar Degas,European Paintings,1867,1868,,Painting,
80,437980,Cypresses,Vincent van Gogh,European Paintings,1889,1889,,Painting,
81,834585,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,
82,834580,"A Life in a Year–The Garden–Noon, from ""Pictur...",Arthur Boyd Houghton,Drawings and Prints,1866,1874,,Print,


In [23]:
# Сколько объектов относятся к японской культуре? (culture –Japan)
df \
    .groupby('culture', as_index = False) \
    .agg({'objectID' : 'count'})

,culture,objectID
0,,72
1,American,9
2,Ancestral Pueblo,1
3,British,1
4,Japan,1


In [36]:
# 5 Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. 
# Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

# Note: выполнение цикла может занять некоторое время (~5 минут), поэтому проверять его работоспособность лучше на более маленькой выборке!
# В качестве ответа укажите, в каком периоде (period) было выполнено наибольшее количество работ, представленных в музее?

r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()


In [34]:
%%time
cats_data=[] #создаем пустой список, из которого будем потом собирать DF

for i in cats['objectIDs']:
    r_cats = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
    obj_cats = r_cats.json()
    if len(cats_data) < 1000:
        try: 
            cats_data +=[ [obj_cats['objectID'], 
                      obj_cats['title'], 
                      obj_cats['artistDisplayName'], 
                      obj_cats['department'], 
                      obj_cats['objectBeginDate'], 
                      obj_cats['objectEndDate'], 
                      obj_cats['period'], 
                      obj_cats['objectName'], 
                      obj_cats['culture']]]
        except: 
            continue
    else: break
cats_df =pd.DataFrame(cats_data, columns = ['objectID', 
                                  'title', 
                                  'artistDisplayName', 
                                  'department',
                                 'objectBeginDate',
                                 'objectEndDate',
                                 'period',
                                 'objectName', 
                                 'culture'])
cats_df.head()

CPU times: user 27.5 s, sys: 1.58 s, total: 29 s
Wall time: 4min 2s


,objectID,title,artistDisplayName,department,objectBeginDate,objectEndDate,period,objectName,culture
0,49698,Cat,Zhang Yuguang,Asian Art,1900,1968,,Folding fan mounted as an album leaf,China
1,49470,Cat,Wang Li,Asian Art,1813,1879,Qing dynasty (1644–1911),Hanging scroll,China
2,53222,Musk Cat,Uto Gyoshi,Asian Art,1550,1599,Muromachi period (1392–1573),Hanging scroll,Japan
3,36221,Cat,Wang Yun,Asian Art,1900,1933,,Hanging scroll,China
4,60873,Burial Crown,,Asian Art,960,1279,Song dynasty (960–1279) or later,Burial crown,China


In [39]:
cats_df \
    .groupby('period', as_index = False) \
    .agg({'objectID' : 'count'}) \
    .sort_values('objectID', ascending = False)

,period,objectID
16,Edo period (1615–1868),441
0,,110
75,Qing dynasty (1644–1911),62
43,Ming dynasty (1368–1644),36
63,Muromachi period (1392–1573),30
...,...,...
51,"Ming dynasty (1368–1644), Wanli mark and perio...",1
50,"Ming dynasty (1368–1644), Tianqi period (1621–27)",1
49,"Ming dynasty (1368–1644), Jiajing period (1522...",1
48,"Ming dynasty (1368–1644), Jiajing period (1522...",1
